# Diffusion-based Super-Resolution for EuroSAT

Training diffusion models for 13-channel multispectral satellite imagery super-resolution.

**Current Best Result:** 19.31 dB PSNR, 0.7379 SSIM (epoch 55)

## Setup

In [5]:
# Install dependencies
!pip install -q torchgeo lpips

In [6]:
import os

if not os.path.exists('diffusion-sr-eurosat'):
    !git clone https://github.com/ParkerJ1/diffusion-sr-eurosat.git

%cd diffusion-sr-eurosat
print("In project directory:", os.getcwd())

!pip install -q torchgeo lpips

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0, '.')

from configs.config import BaseConfig
from src.data.dataset import get_dataloader
from src.models.unet import ConditionalUNet
from src.models.diffusion import Scheduler
from src.training.train import train
from src.training.sample import sampling
from src.evaluation.metrics import metrics
from src.utils.visualisation import visualize_and_save_samples
from src.utils.helpers import find_latest_model

Cloning into 'diffusion-sr-eurosat'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 37 (delta 3), reused 32 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 34.22 KiB | 3.80 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/diffusion-sr-eurosat/diffusion-sr-eurosat
In project directory: /content/diffusion-sr-eurosat/diffusion-sr-eurosat
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Imports

In [7]:
import torch
import torch.optim as optim
from tqdm import tqdm
import os

# Import from our modules
from configs.config import BaseConfig
from src.data.dataset import get_dataloader
from src.models.unet import ConditionalUNet
from src.models.diffusion import Scheduler
from src.training.train import train
from src.training.sample import sampling
from src.evaluation.metrics import metrics
from src.utils.visualisation import visualize_and_save_samples
from src.utils.helpers import find_latest_model

print("All imports successful!")

ModuleNotFoundError: No module named 'src.utils.visualization'

## Configuration

In [ ]:
# Create configuration
config = BaseConfig()

# Override any settings for this experiment
# config.EPOCHS = 200
# config.BATCH_SIZE = 256
# config.LR = 1e-4

print(f"Device: {config.DEVICE}")
print(f"Dataset: {config.DATASET}")
print(f"Batch Size: {config.BATCH_SIZE}")
print(f"Epochs: {config.EPOCHS}")
print(f"Learning Rate: {config.LR}")
print(f"Output Directory: {config.OUTPUT_DIR}")

## Initialize Model & Data

In [ ]:
# Create output directories
if config.TRAIN:
    os.makedirs(config.OUTPUT_DIR, exist_ok=True)
    os.makedirs(config.DATASET_PATH, exist_ok=True)
    print(f"Output directory: {config.OUTPUT_DIR}")

# Get dataloader
dataloader = get_dataloader(config)
print(f"✅ Dataloader created: {len(dataloader)} batches")

# Get validation batch
val_high_res_img, val_low_res_img = next(iter(dataloader))
val_high_res_img = val_high_res_img[:config.NUM_EVAL_SAMPLES].to(config.DEVICE)
val_low_res_img = val_low_res_img[:config.NUM_EVAL_SAMPLES].to(config.DEVICE)
print(f"✅ Validation batch: {val_high_res_img.shape}")

In [ ]:
# Initialize model
model = ConditionalUNet(config).to(config.DEVICE)

# Initialize optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=config.LR,
    weight_decay=config.WEIGHT_DECAY
)

# Initialize scheduler
scheduler = Scheduler()
alphas_cumprod = scheduler.get_linear_beta_scheduler(
    config.TIMESTEPS,
    config.DEVICE
)

# Initialize metrics
psnr_metric = metrics.PSNR(data_range=2.0)  # [-1, 1] range
ssim_metric = metrics.SSIM(data_range=2.0)
lpips_metric = metrics.LPIPS()

print(f"✅ Model initialized: {sum(p.numel() for p in model.parameters()):,} parameters")
print(f"✅ Optimizer: AdamW (lr={config.LR})")
print(f"✅ Scheduler: Linear beta ({config.TIMESTEPS} timesteps)")

In [ ]:
# Load existing model if in inference mode
if not config.TRAIN:
    if config.MODEL_PATH is None:
        # Auto-detect latest model
        config.MODEL_PATH = find_latest_model(config.PROJECT_ROOT)

    if config.MODEL_PATH and os.path.exists(config.MODEL_PATH):
        checkpoint = torch.load(config.MODEL_PATH, map_location=config.DEVICE)
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"✅ Loaded model from: {config.MODEL_PATH}")
    else:
        print("⚠️  No model found, starting from scratch")
        config.TRAIN = True

## Training Loop

In [ ]:
if config.TRAIN:
    print("Starting training...")
    print("=" * 60)

    for epoch in range(config.EPOCHS):
        print(f"\n--- Epoch {epoch+1}/{config.EPOCHS} ---")

        # Train for one epoch
        avg_loss = train(
            model=model,
            dataloader=dataloader,
            scheduler=scheduler,
            optimizer=optimizer,
            device=config.DEVICE,
            epoch=epoch
        )

        print(f"Epoch: {epoch+1} - Average loss: {avg_loss:.4f}")

        # Sample and evaluate
        if (epoch + 1) % config.SAMPLE_EVERY == 0:
            print(f"---Sampling - epoch {epoch+1}")

            # Generate samples
            generated_images = sampling(
                model=model,
                low_res_imgs=val_low_res_img,
                scheduler=scheduler,
                config=config
            )

            # Compute metrics
            psnr = psnr_metric(generated_images, val_high_res_img)
            ssim = ssim_metric(generated_images, val_high_res_img)
            lpips_score = lpips_metric(generated_images, val_high_res_img)

            print(f"PSNR: {psnr:.2f} dB | SSIM: {ssim:.4f} | LPIPS: {lpips_score:.4f}")

            # Save samples and visualizations
            visualize_and_save_samples(
                generated_images=generated_images,
                val_high_res_img=val_high_res_img,
                val_low_res_img=val_low_res_img,
                epoch=epoch+1,
                config=config
            )

            # Save checkpoint
            checkpoint_path = os.path.join(config.OUTPUT_DIR, f'model_epoch_{epoch+1}.pth')
            torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
                'psnr': psnr,
                'ssim': ssim,
                'lpips': lpips_score,
            }, checkpoint_path)
            print(f"Checkpoint saved: {checkpoint_path}")

    # Save final model
    final_model_path = os.path.join(config.OUTPUT_DIR, 'model.pth')
    torch.save({
        'epoch': config.EPOCHS,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, final_model_path)
    print(f"\n✅ Training complete! Final model saved: {final_model_path}")

else:
    print("Skipping training (config.TRAIN = False)")

## Evaluation / Inference

In [ ]:
# Generate samples (works in both training and inference mode)
print("Generating samples...")

generated_images = sampling(
    model=model,
    low_res_imgs=val_low_res_img,
    scheduler=scheduler,
    config=config
)

# Compute metrics
psnr = psnr_metric(generated_images, val_high_res_img)
ssim = ssim_metric(generated_images, val_high_res_img)
lpips_score = lpips_metric(generated_images, val_high_res_img)

print(f"\n📊 Evaluation Results:")
print(f"PSNR:  {psnr:.2f} dB")
print(f"SSIM:  {ssim:.4f}")
print(f"LPIPS: {lpips_score:.4f}")

# Visualize
visualize_and_save_samples(
    generated_images=generated_images,
    val_high_res_img=val_high_res_img,
    val_low_res_img=val_low_res_img,
    epoch='final',
    config=config
)

## Additional Analysis (Optional)

In [ ]:
# Evaluate on larger test set (optional)
num_test_samples = 32

print(f"Evaluating on {num_test_samples} samples...")

all_psnr = []
all_ssim = []
all_lpips = []

for i, (high_res, low_res) in enumerate(dataloader):
    if i >= num_test_samples // config.BATCH_SIZE:
        break

    high_res = high_res.to(config.DEVICE)
    low_res = low_res.to(config.DEVICE)

    with torch.no_grad():
        generated = sampling(model, low_res, scheduler, config)

    all_psnr.append(psnr_metric(generated, high_res))
    all_ssim.append(ssim_metric(generated, high_res))
    all_lpips.append(lpips_metric(generated, high_res))

import numpy as np
print(f"\n📊 Results on {num_test_samples} samples:")
print(f"PSNR:  {np.mean(all_psnr):.2f} ± {np.std(all_psnr):.2f} dB")
print(f"SSIM:  {np.mean(all_ssim):.4f} ± {np.std(all_ssim):.4f}")
print(f"LPIPS: {np.mean(all_lpips):.4f} ± {np.std(all_lpips):.4f}")